In [ ]:
# Azure NSG (Network Security Group) Rules JSON to Excel coverter

# --- Colab Setup ---
!pip install pandas openpyxl

import json, os, re
import pandas as pd
from google.colab import files
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

# --- Region Formatter (Universal) ---
def format_location(loc: str) -> str:
    if not loc:
        return ""
    loc_lower = loc.lower()
    region_map = {  # abbreviated list; see full universal map in previous step
        "australiaeast":"Australia East","australiasoutheast":"Australia Southeast",
        "eastus":"East US","eastus2":"East US 2","westeurope":"West Europe",
        "northeurope":"North Europe","uksouth":"UK South","ukwest":"UK West",
        "southeastasia":"Southeast Asia","centralus":"Central US","southcentralus":"South Central US",
        "westus":"West US","westus2":"West US 2","canadacentral":"Canada Central",
        "canadaeast":"Canada East","brazilsouth":"Brazil South","japaneast":"Japan East",
        "japanwest":"Japan West","francecentral":"France Central","germanywestcentral":"Germany West Central"
    }
    if loc_lower in region_map: return region_map[loc_lower]
    return re.sub(r'([a-z])([A-Z0-9])', r'\1 \2', loc_lower.title()).replace("-", " ").title()

# --- Upload JSON ---
print("📤 Upload your NSG JSON file:")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# --- Parse JSON ---
with open(filename,'r') as f: data=json.load(f)
nsg_name=data.get("name","Unknown NSG")
id_path=data.get("id","")
subscription_id=id_path.split("/")[2] if id_path else ""
resource_group=id_path.split("/resourceGroups/")[1].split("/")[0] if "/resourceGroups/" in id_path else ""
metadata={
 "Resource group":resource_group,
 "Location":format_location(data.get("location","")),
 "Subscription ID":subscription_id
}

# --- Collect Rules ---
rules=data["properties"].get("securityRules",[])+data["properties"].get("defaultSecurityRules",[])
def replace_any(v):
  if isinstance(v,str): return "Any" if v.strip()=="*" else v
  if isinstance(v,list): return ", ".join(replace_any(x) for x in v if x)
  return v
records=[]
for r in rules:
  p=r.get("properties",{})
  ports=p.get("destinationPortRanges",[]) or [p.get("destinationPortRange","Any")]
  sources=p.get("sourceAddressPrefixes",[]) or [p.get("sourceAddressPrefix","Any")]
  dests=p.get("destinationAddressPrefixes",[]) or [p.get("destinationAddressPrefix","Any")]
  records.append({
    "Priority":int(p.get("priority",0)),
    "Direction":p.get("direction",""),
    "RuleName":r.get("name",""),
    "Port":replace_any(ports),
    "Protocol":replace_any(p.get("protocol","")),
    "Source":replace_any(sources),
    "Destination":replace_any(dests),
    "Access":p.get("access",""),
    "Description":p.get("description","")
  })
df=pd.DataFrame(records)
df["Direction"]=pd.Categorical(df["Direction"],categories=["Inbound","Outbound"],ordered=True)
df=df.sort_values(["Direction","Priority"])

# --- Excel Styling ---
wb=Workbook(); ws=wb.active; ws.title="NSG_RULES"
bold, title=Font(bold=True), Font(bold=True,size=14)
fill_hdr=PatternFill("solid","D9E1F2"); fill_title=PatternFill("solid","BDD7EE")
align_center=Alignment(horizontal="center")
border=Border(left=Side(style="thin"),right=Side(style="thin"),top=Side(style="thin"),bottom=Side(style="thin"))
cols=["Priority","Direction","RuleName","Port","Protocol","Source","Destination","Access","Description"]

ws.append([nsg_name]); ws.merge_cells(start_row=1,start_column=1,end_row=1,end_column=len(cols))
ws["A1"].font=title; ws["A1"].fill=fill_title; ws["A1"].alignment=align_center
ws.append([""])
for k,v in metadata.items(): ws.append([k,v]); ws[f"A{ws.max_row}"].font=bold
ws.append([""])
ws.append(cols)
for i,c in enumerate(cols,1):
  cell=ws[f"{get_column_letter(i)}{ws.max_row}"]; cell.font=bold; cell.fill=fill_hdr
for _,r in df.iterrows(): ws.append([r.get(c,"") for c in cols])
for row in ws.iter_rows(min_row=1,max_row=ws.max_row):
  for c in row:
    if c.value: c.border=border
for col in ws.columns:
  ws.column_dimensions[get_column_letter(col[0].column)].width=max(len(str(c.value)) for c in col if c.value)+3
out=os.path.splitext(filename)[0]+".xlsx"; wb.save(out)
files.download(out)
